In [9]:
#Load the fine-tuned BART model
from transformers import BartForSequenceClassification

model = BartForSequenceClassification.from_pretrained("../BART_model", problem_type="multi_label_classification", num_labels=2)

loading configuration file ../BART_model\config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2labe

In [10]:
#Load the tokenizer
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("../BART_model")

Didn't find file ../BART_model\added_tokens.json. We won't load it.
loading file ../BART_model\vocab.json
loading file ../BART_model\merges.txt
loading file None
loading file ../BART_model\special_tokens_map.json
loading file ../BART_model\tokenizer_config.json


In [3]:
#load original data
import pandas as pd 

df = pd.read_csv('../Data/webmd.csv')
df['Date'] = df['Date'].astype('datetime64[ns]')
df = df.drop_duplicates('Reviews')
df = df[df['Condition'].str.contains("osteoporosis", case = False)]
df

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
4147,25-34,Osteoporosis,2010-03-16,lotensin,6883,4,3,this treatment is awsome. I love drugs.,2,Male,"Dizziness , lightheadedness , drowsiness, or ...",0
6192,45-54,Post-Menopausal Osteoporosis Prevention,2017-07-31,lopreeza,167327,5,2,After taking this drug for approx. 21 days I s...,3,Male,"Stomach upset, nausea / vomiting , bloating...",0
6207,45-54,Post-Menopausal Osteoporosis Prevention,2016-12-29,lopreeza,167327,5,5,I have taken this drug for almost 7 years with...,5,Female,"Stomach upset, nausea / vomiting , bloating...",1
18993,65-74,Osteoporosis,2010-05-26,liquid calcium + vitamin d,93396,5,5,since I have a hard time swallowing this hs wo...,5,Female,Constipation or stomach upset may occur.,1
61145,45-54,Osteoporosis,2012-01-19,oyster shell + d,94390,1,1,I have severe pain in my hand and muscle joint...,1,Female,Constipation or stomach upset may occur.,2
...,...,...,...,...,...,...,...,...,...,...,...,...
360231,55-64,Osteoporosis,2010-11-15,calcium 600 + vitamin d,93224,4,5,The only thing I have to say against these tab...,4,Female,Constipation or stomach upset may occur.,11
360237,55-64,Osteoporosis,2016-03-16,calcium 500 + vitamin d,151589,4,4,Just started taking this a month ago. Definite...,4,,Constipation or stomach upset may occur.,1
360264,55-64,Osteoporosis,2012-03-08,calcitrate + vit d,19844,3,3,"This supplement has caused much gas, bloating,...",1,Female,Constipation or stomach upset may occur.,2
360298,65-74,Osteoporosis,2012-07-24,calcitrate,22129,1,1,first dose: one tablet knocked me out cold for...,1,Female,Constipation and upset stomach may occur.,1


In [48]:
df = df.reset_index().drop(columns=["index"])
df


,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount
0,25-34,Osteoporosis,2010-03-16,lotensin,6883,4,3,this treatment is awsome. I love drugs.,2,Male,"Dizziness , lightheadedness , drowsiness, or ...",0
1,45-54,Post-Menopausal Osteoporosis Prevention,2017-07-31,lopreeza,167327,5,2,After taking this drug for approx. 21 days I s...,3,Male,"Stomach upset, nausea / vomiting , bloating...",0
2,45-54,Post-Menopausal Osteoporosis Prevention,2016-12-29,lopreeza,167327,5,5,I have taken this drug for almost 7 years with...,5,Female,"Stomach upset, nausea / vomiting , bloating...",1
3,65-74,Osteoporosis,2010-05-26,liquid calcium + vitamin d,93396,5,5,since I have a hard time swallowing this hs wo...,5,Female,Constipation or stomach upset may occur.,1
4,45-54,Osteoporosis,2012-01-19,oyster shell + d,94390,1,1,I have severe pain in my hand and muscle joint...,1,Female,Constipation or stomach upset may occur.,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1352,55-64,Osteoporosis,2010-11-15,calcium 600 + vitamin d,93224,4,5,The only thing I have to say against these tab...,4,Female,Constipation or stomach upset may occur.,11
1353,55-64,Osteoporosis,2016-03-16,calcium 500 + vitamin d,151589,4,4,Just started taking this a month ago. Definite...,4,,Constipation or stomach upset may occur.,1
1354,55-64,Osteoporosis,2012-03-08,calcitrate + vit d,19844,3,3,"This supplement has caused much gas, bloating,...",1,Female,Constipation or stomach upset may occur.,2
1355,65-74,Osteoporosis,2012-07-24,calcitrate,22129,1,1,first dose: one tablet knocked me out cold for...,1,Female,Constipation and upset stomach may occur.,1


In [49]:
#make inference
import torch

sigmoid = torch.nn.Sigmoid()
side_effect = []
effectiveness = []
for i in range(len(df["Reviews"])):
    text = df["Reviews"][i]
    encoding = tokenizer(text, return_tensors="pt")
    outputs = model(**encoding)
    probs = sigmoid(outputs.logits.squeeze().cpu()).detach().numpy()
    side_effect.append(probs[0])#return the probabilities
    effectiveness.append(probs[1])

In [50]:
df["side_effect"] = side_effect
df["effectiveness"] = effectiveness

In [51]:
df

,Age,Condition,Date,Drug,DrugId,EaseofUse,Effectiveness,Reviews,Satisfaction,Sex,Sides,UsefulCount,side_effect,effectiveness
0,25-34,Osteoporosis,2010-03-16,lotensin,6883,4,3,this treatment is awsome. I love drugs.,2,Male,"Dizziness , lightheadedness , drowsiness, or ...",0,0.001712,0.001131
1,45-54,Post-Menopausal Osteoporosis Prevention,2017-07-31,lopreeza,167327,5,2,After taking this drug for approx. 21 days I s...,3,Male,"Stomach upset, nausea / vomiting , bloating...",0,0.999636,0.000561
2,45-54,Post-Menopausal Osteoporosis Prevention,2016-12-29,lopreeza,167327,5,5,I have taken this drug for almost 7 years with...,5,Female,"Stomach upset, nausea / vomiting , bloating...",1,0.999462,0.999086
3,65-74,Osteoporosis,2010-05-26,liquid calcium + vitamin d,93396,5,5,since I have a hard time swallowing this hs wo...,5,Female,Constipation or stomach upset may occur.,1,0.001908,0.998169
4,45-54,Osteoporosis,2012-01-19,oyster shell + d,94390,1,1,I have severe pain in my hand and muscle joint...,1,Female,Constipation or stomach upset may occur.,2,0.999454,0.000543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,55-64,Osteoporosis,2010-11-15,calcium 600 + vitamin d,93224,4,5,The only thing I have to say against these tab...,4,Female,Constipation or stomach upset may occur.,11,0.994593,0.004499
1353,55-64,Osteoporosis,2016-03-16,calcium 500 + vitamin d,151589,4,4,Just started taking this a month ago. Definite...,4,,Constipation or stomach upset may occur.,1,0.999819,0.016065
1354,55-64,Osteoporosis,2012-03-08,calcitrate + vit d,19844,3,3,"This supplement has caused much gas, bloating,...",1,Female,Constipation or stomach upset may occur.,2,0.999842,0.003682
1355,65-74,Osteoporosis,2012-07-24,calcitrate,22129,1,1,first dose: one tablet knocked me out cold for...,1,Female,Constipation and upset stomach may occur.,1,0.999637,0.000877


In [ ]:
df.to_csv("preprocessed_osteoporosis_prob.csv")